In [1]:
import nltk
from nltk.corpus.reader.plaintext import *
#from nltk.corpus import stopwords
import collections
from nltk.util import ngrams

from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns

import os

In [2]:
corpus = PlaintextCorpusReader('Z:\\Documents\\Son Reader\\Corpus', '.*\.txt')

#### first, lets pull some sentences about Sprint and he how each model performs

In [3]:
def wordSentCount (text, word, verbose=False):
    
    sents = sent_tokenize(corpus.raw(text).lower())
    sent_list = []
                          
    for sent in sents:
        for item in word_tokenize(sent):
            if item == word:
                if verbose:
                    print (">>>>> Just found ") + str(item) + str(" in ") + str(text) 
                if sent not in sent_list:
                    sent_list.append(sent)
    return sent_list


In [4]:
def bigramSentCount (text, bigram, verbose=False):
    
    sents = sent_tokenize(corpus.raw(text).lower())
    sent_list = []
    
    for sent in sents:
        pairs = [ " ".join(pair) for pair in nltk.bigrams(word_tokenize(sent))]
        for item in pairs:
            if item == bigram:
                if verbose:
                    print (">>>>> Just found ") + str(item) + str(" in ") + str(text) 
                if sent not in sent_list:
                    sent_list.append(sent)
    return sent_list

In [5]:
def trigramSentCount (text, trigram, verbose=False):

    sents = sent_tokenize(corpus.raw(text).lower())
    sent_list = []
    
    for sent in sents:
        pairs = [ " ".join(pair) for pair in nltk.trigrams(word_tokenize(sent))]
        for item in pairs:
            if item == trigram:
                if verbose:
                    print (">>>>> Just found ") + str(item) + str(" in ") + str(text) 
                if sent not in sent_list:
                    sent_list.append(sent)
    return sent_list

In [6]:
def countsBySentence (corpus, keyword_cohort, verbose=False):

    # keyword_cohort is a list of single-, two- and three-word keywords
    # corpus is a NLTK collection of text files
    
    # the final dictionary in the form of {"Date":  [sents]} 
    sentences = collections.OrderedDict()
    for fname in corpus.fileids():
        sentences[fname[:7]] = []

   
    #cycles through keywords in a cohort
    for keyword in keyword_cohort:
        if verbose:
            print (">>> Now counting ") + str(keyword)

        # if the keyword is a trigram
        if len(word_tokenize(keyword))==3:
            #cycles through the corpus, finding totals for each documents
            for fname in corpus.fileids():
                this_sent_list = trigramSentCount(fname, keyword, verbose = verbose)
                sentences[fname[:7]].extend(this_sent_list)

        # if the keyword is a bigram
        elif len(word_tokenize(keyword))==2:
            #cycles through the corpus, finding totals for each documents
            for fname in corpus.fileids():
                this_sent_list = bigramSentCount(fname, keyword, verbose = verbose)
                sentences[fname[:7]].extend(this_sent_list)

        #if the keyword is a single  
        else:
            for fname in corpus.fileids():
                this_sent_list = wordSentCount(fname, keyword, verbose = verbose)
                sentences [fname[:7]].extend(this_sent_list)
   
           
    return sentences

In [7]:
keyword_cohort = ["sprint", "marcelo", "t-mobile", "at&t", "verizon", "deutsche telekom"] 

In [8]:
sentences = countsBySentence(corpus, keyword_cohort, verbose=False)
sentences

OrderedDict([('2006 Q1', []),
             ('2006 Q2', []),
             ('2006 Q3', []),
             ('2006 Q4', []),
             ('2007 Q2', []),
             ('2007 Q3', []),
             ('2007 Q4', []),
             ('2008 Q1', []),
             ('2008 Q2', []),
             ('2008 Q3', []),
             ('2008 Q4', []),
             ('2009 Q1',
              [u"so that's why that we set up the joint venture with the verizon, vodafone\r\nand china mobile so that we'd be able to increase that area."]),
             ('2009 Q2', []),
             ('2009 Q3', []),
             ('2009 Q4',
              [u"first is that is shown on this slide, docomo's data arpu ratio is 44%, sk telecom the number one in the south\r\nkorea is at 35%, singtel 32%, and the u.s. biggest mobile operator verizon wireless is at 32%."]),
             ('2010 Q1',
              [u"this is amongst softbank, china\r\nmobile, vodafone and verizon, so it's those four parties that we have an alliance to build the 

In [31]:
corpus = []
sents = [corpus.extend(item) for item in sentences.values() if item != []]
corpus

[u"so that's why that we set up the joint venture with the verizon, vodafone\r\nand china mobile so that we'd be able to increase that area.",
 u"first is that is shown on this slide, docomo's data arpu ratio is 44%, sk telecom the number one in the south\r\nkorea is at 35%, singtel 32%, and the u.s. biggest mobile operator verizon wireless is at 32%.",
 u"this is amongst softbank, china\r\nmobile, vodafone and verizon, so it's those four parties that we have an alliance to build the business for mobile phone\r\nand develop the technology.",
 u'so data arpu ratio we have already exceeded 50% and now reaching to 58% which is the number one in the world\r\ncompared to vodafone, singtel, verizon.',
 u'and verizon in the\r\nu.s., their operating income grew just 0.3%.',
 u'those big players in the us, at&t and verizon, it was 2% or negative 5%.',
 u'inc.\r\nnow the other day we announced a deal with sprint and a lot of people said that softbank is going to heavily\r\nindebted it and softba

#### first, try google API

In [10]:
def gc_sentiment(text):  
    from google.cloud import language
    
    client = language.LanguageServiceClient()
    document = language.types.Document(
            content=text,
            type=language.enums.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(document=document)
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    return score, magnitude

In [11]:
from tqdm import tqdm
gc_results = [gc_sentiment(row) for row in tqdm(corpus, ncols = 100)]
gc_score, gc_magnitude = zip(*gc_results) # Unpacking the result into 2 lists
gc = list(zip(corpus, gc_score, gc_magnitude))
columns = ['text', 'score', 'magnitude']
gc_df = pd.DataFrame(gc, columns = columns)

  0%|                                                                       | 0/693 [00:00<?, ?it/s]


ImportError: No module named google.cloud

#### fuck it, we'll do Google API is all else fails....

#### first, lets clean up the text

In [33]:
corpus[0].replace("\r\n", " ")

u"so that's why that we set up the joint venture with the verizon, vodafone and china mobile so that we'd be able to increase that area."

In [35]:
corpus = [item.replace("\r\n", " ") for item in corpus]
corpus[0]

u"so that's why that we set up the joint venture with the verizon, vodafone and china mobile so that we'd be able to increase that area."

#### lets give NLTK a shot

In [36]:
def nltk_sentiment(sentence):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    
    nltk_sentiment = SentimentIntensityAnalyzer()
    score = nltk_sentiment.polarity_scores(sentence)
    return score

In [37]:
nltk_results = [nltk_sentiment(row) for row in corpus]
results_df = pd.DataFrame(nltk_results)
text_df = pd.DataFrame(corpus, columns = ['text'])
nltk_df = text_df.join(results_df)

In [38]:
nltk_df

,text,compound,neg,neu,pos
0,so that's why that we set up the joint venture...,0.3182,0.000,0.916,0.084
1,"first is that is shown on this slide, docomo's...",0.0772,0.000,0.967,0.033
2,"this is amongst softbank, china mobile, vodafo...",0.4019,0.000,0.915,0.085
3,so data arpu ratio we have already exceeded 50...,0.2732,0.000,0.890,0.110
4,"and verizon in the u.s., their operating incom...",0.0000,0.000,1.000,0.000
5,"those big players in the us, at&t and verizon,...",-0.5719,0.209,0.791,0.000
6,inc. now the other day we announced a deal wit...,-0.2732,0.070,0.930,0.000
7,in continuing revenue and profit increase and ...,0.6369,0.000,0.877,0.123
8,but why sprint?,0.0000,0.000,1.000,0.000
9,because we see high growth potential in sprint...,0.8625,0.000,0.714,0.286


In [39]:
nltk_df.describe()

,compound,neg,neu,pos
count,693.000000,693.000000,693.000000,693.000000
mean,0.184310,0.032446,0.864371,0.103182
std,0.378681,0.074941,0.127847,0.120513
min,-0.833800,0.000000,0.303000,0.000000
25%,0.000000,0.000000,0.779000,0.000000
50%,0.051600,0.000000,0.872000,0.075000
75%,0.467800,0.000000,1.000000,0.180000
max,0.972100,0.508000,1.000000,0.697000


In [40]:
trimmed = nltk_df.loc[:, ["text", "compound"]]
trimmed.head()

,text,compound
0,so that's why that we set up the joint venture...,0.3182
1,"first is that is shown on this slide, docomo's...",0.0772
2,"this is amongst softbank, china mobile, vodafo...",0.4019
3,so data arpu ratio we have already exceeded 50...,0.2732
4,"and verizon in the u.s., their operating incom...",0.0000


In [44]:
trimmed.columns = ["text", "score"]

In [45]:
trimmed.head()

,text,score
0,so that's why that we set up the joint venture...,0.3182
1,"first is that is shown on this slide, docomo's...",0.0772
2,"this is amongst softbank, china mobile, vodafo...",0.4019
3,so data arpu ratio we have already exceeded 50...,0.2732
4,"and verizon in the u.s., their operating incom...",0.0000


In [66]:
neutral = trimmed[trimmed.score == 0.0000]
negative = trimmed[trimmed.score < 0.0000].sort_values(by="score")
positive = trimmed[trimmed.score > 0.0000].sort_values(by="score")

In [67]:
negative.head()

,text,score
176,"but as you can see here, the sprint booked net...",-0.8338
458,"so, before, there were a lot of negative media...",-0.8126
175,"but whatever the accounting principle is, mayb...",-0.7964
32,so recently there is some mass media coverage ...,-0.7783
567,"about a year ago, at that time, sprint was rec...",-0.7783


In [70]:
worst_five = negative.iloc[0:5,0]
for text in worst_five:
    print text

but as you can see here, the sprint booked net loss for the past years and that this loss for the current period include the impairment loss.
so, before, there were a lot of negative media coverage and sprint has always been negative part of softbank.
but whatever the accounting principle is, maybe we should accept humbly the fact that the sprint booked the impairment loss, and although we didn't book the impairment loss on a consolidated basis, we should pretend that we suffered a loss, so we should try to pursue further efficiency.
so recently there is some mass media coverage talking about the negative figures about sprint and there was worse on the media.
about a year ago, at that time, sprint was recording red and still those moment that the people were worried and anxious about sprint may go bankrupt.


In [71]:
best_five = positive.iloc[0:5,0]
for text in best_five:
    print text

now i see the improvements in sprint network every day and that resulted in lower churn rate.
and for the global, or business-wise, we have also appointed as the vice chairman of softbank corp., and who used to be a former senior vice president and chief business officer of google, mr. nikesh arora, assumed the vice chairman of softbank corp. so in a business – he was the chief business officer of google inc. and mr. nikesh arora assumed the position of vice chairman of softbank corp. he was the number two in google, which is one of the largest internet companies, so he has well-known about internet industry but before then, he was also involved in the mobile business of deutsche telekom, so he has communication knowledge and internet knowledge, so he has both knowledge and skill set which is very rare personnel in the world.
so, network has been improving a lot, that's why sprint churn level is the lowest in sprint history.
and having a difficulty in sprint and having a stable stage i

#### not too bad?

In [72]:
positive.iloc[0,0]

u'now i see the improvements in sprint network every day and that resulted in lower churn rate.'